In [2]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import os
import pickle
import torch
from collections import Counter
from dynamic_batch_size import dynamic_batch_size

c:\Users\matth\miniconda3\envs\embeddings\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('works-2023-onward-quantum-10-citations-article-open-access.csv')

In [4]:
df_filtered = df.dropna(subset='abstract')

In [5]:
df_filtered.shape

(1141, 179)

In [7]:
mistral = SentenceTransformer("zeta-alpha-ai/Zeta-Alpha-E5-Mistral",trust_remote_code=True)
mistral.to(torch.float16)

Loading checkpoint shards: 100%|██████████| 3/3 [00:04<00:00,  1.57s/it]


SentenceTransformer(
  (0): Transformer({'max_seq_length': 4096, 'do_lower_case': False}) with Transformer model: MistralModel 
  (1): Pooling({'word_embedding_dimension': 4096, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': True, 'include_prompt': True})
  (2): Normalize()
)

In [9]:
start = 0
while start < df_filtered.shape[0]:
    batch_size = dynamic_batch_size(df_filtered['abstract'].values[start:],
                                    50000,
                                    1/3)
    print(start,batch_size)
    temp = mistral.encode(df_filtered['abstract'].values[start:start+batch_size],
                          normalize_embeddings=True,)
    with open(f'vec/{start}-{start+batch_size}.pkl','wb') as f:
        pickle.dump(temp,f)
    start += batch_size

0 38
38 38
76 41
117 42
159 43
202 41
243 39
282 39
321 39
360 41
401 36
437 37
474 42
516 40
556 47
603 46
649 41
690 40
730 38
768 42
810 38
848 40
888 41
929 39
968 37
1005 35
1040 31
1071 29
1100 24
1124 16
1140 1
